In [4]:
import pandas as pd
import datetime
import numpy as np

import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
import jupyter_dash
from dash.dependencies import Input, Output
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from IPython.display import display

import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats
import pingouin

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

In [5]:
# pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)

In [6]:
experiment2 = pd.read_csv('data/experiment2.csv')
experiment2 = experiment2[14:]
experiment2 = experiment2[:85]
experiment2

,Date,Temperature,Respiration Rate,Vitals Heart Rate,Blood Pressure Systolic,Blood Pressure Diastolic,Mean Arterial Pressure,Step Count,Resting Heart Rate,Number of Floors,Heart Rate Count,Heart Rate Mean,Heart Rate STD,Heart Rate Minimum,Heart Rate 25%,Heart Rate Median,Heart Rate 75%,Heart Rate Maximum,Mood,Pain,Tired,Sick,Anxiousness,Depressed,Suicidal,Self-Harm,Thoughts of Death,Repetitive Thoughts,Thoughts of Worthlessness,Thoughts of Hopelessness,Brain Sludge,Sluggish,Restless,Existential Crisis,Loss of Interest,Self Isolation,Paranoia,Elevated,Irritable/Angry,Loneliness,Libido,Weight,Weird,sleeping_durations_seconds,sleeping_counts,sleeping_durations_seconds_7_day_rolling_mean,internet_durations_seconds,internet_counts,internet_durations_seconds_7_day_rolling_mean,working_durations_seconds,working_counts,working_durations_seconds_7_day_rolling_mean,games_durations_seconds,games_counts,games_durations_seconds_7_day_rolling_mean,family_time_durations_seconds,family_time_counts,family_time_durations_seconds_7_day_rolling_mean,eating_durations_seconds,eating_counts,eating_durations_seconds_7_day_rolling_mean,programming_durations_seconds,programming_counts,programming_durations_seconds_7_day_rolling_mean,grooming_durations_seconds,grooming_counts,grooming_durations_seconds_7_day_rolling_mean,reading_durations_seconds,reading_counts,reading_durations_seconds_7_day_rolling_mean,gym_durations_seconds,gym_counts,gym_durations_seconds_7_day_rolling_mean,toilet_durations_seconds,toilet_counts,toilet_durations_seconds_7_day_rolling_mean,friends_time_durations_seconds,friends_time_counts,friends_time_durations_seconds_7_day_rolling_mean,studying_durations_seconds,studying_counts,studying_durations_seconds_7_day_rolling_mean,movies_tv_durations_seconds,movies_tv_counts,movies_tv_durations_seconds_7_day_rolling_mean,driving_durations_seconds,driving_counts,driving_durations_seconds_7_day_rolling_mean,walking_durations_seconds,walking_counts,walking_durations_seconds_7_day_rolling_mean,writing_durations_seconds,writing_counts,writing_durations_seconds_7_day_rolling_mean,cooking_durations_seconds,cooking_counts,cooking_durations_seconds_7_day_rolling_mean,class_durations_seconds,class_counts,class_durations_seconds_7_day_rolling_mean,cleaning_durations_seconds,cleaning_counts,cleaning_durations_seconds_7_day_rolling_mean,dishes_durations_seconds,dishes_counts,dishes_durations_seconds_7_day_rolling_mean,personal_durations_seconds,personal_counts,personal_durations_seconds_7_day_rolling_mean,dating_durations_seconds,dating_counts,dating_durations_seconds_7_day_rolling_mean,shopping_durations_seconds,shopping_counts,shopping_durations_seconds_7_day_rolling_mean,mycology_durations_seconds,mycology_counts,mycology_durations_seconds_7_day_rolling_mean,errands_and_appointments_durations_seconds,errands_and_appointments_counts,errands_and_appointments_durations_seconds_7_day_rolling_mean,club_social_group_durations_seconds,club_social_group_counts,club_social_group_durations_seconds_7_day_rolling_mean,drinking_durations_seconds,drinking_counts,drinking_durations_seconds_7_day_rolling_mean,gamedev_durations_seconds,gamedev_counts,gamedev_durations_seconds_7_day_rolling_mean,swimming_tanning_durations_seconds,swimming_tanning_counts,swimming_tanning_durations_seconds_7_day_rolling_mean,phone_durations_seconds,phone_counts,phone_durations_seconds_7_day_rolling_mean,keyboard_durations_seconds,keyboard_counts,keyboard_durations_seconds_7_day_rolling_mean,journaling_durations_seconds,journaling_counts,journaling_durations_seconds_7_day_rolling_mean,drawing_durations_seconds,drawing_counts,drawing_durations_seconds_7_day_rolling_mean,traveling__durations_seconds,traveling__counts,traveling__durations_seconds_7_day_rolling_mean,drumming_durations_seconds,drumming_counts,drumming_durations_seconds_7_day_rolling_mean,data_analysis_durations_seconds,data_analysis_counts,data_analysis_durations_seconds_7_day_rolling_mean,concert_event_durations_seconds,co

# Power Analysis for ANOVARM

In [285]:
# df1 = experiment2[['Mood', 'journaling_counts']]
df1 = experiment2['Mood']
df1

14    3.0
15    3.0
16    3.0
17    3.0
18    2.0
     ... 
94    2.0
95    2.0
96    2.0
97    2.0
98    2.0
Name: Mood, Length: 85, dtype: float64

In [286]:
B1 = df1[0:14]
B1 = B1.reset_index().drop('index', axis=1)
B1.rename({'Mood': 'Mood_B', 'journaling_counts':'journaling_counts_B1'}, axis=1, inplace=True)
# B1 = B1.drop(columns='journaling_counts_B1')
B1

,Mood_B
0,3.0
1,3.0
2,3.0
3,3.0
4,2.0
5,2.0
6,2.0
7,3.0
8,3.0
9,3.0


In [287]:
A1 = df1[14:28]
A1 = A1.reset_index().drop('index', axis=1)
A1.rename({'Mood': 'Mood_A', 'journaling_counts':'journaling_counts_A1'}, axis=1, inplace=True)
# A1 = A1.drop(columns='journaling_counts_A1')
A1

,Mood_A
0,2.0
1,2.0
2,2.0
3,2.0
4,2.0
5,2.0
6,2.0
7,3.0
8,3.0
9,2.0


In [288]:
B2 = df1[28:42]
B2 = B2.reset_index().drop('index', axis=1)
B2.rename({'Mood': 'Mood_B', 'journaling_counts':'journaling_counts_B2'}, axis=1, inplace=True)
# B2 = B2.drop(columns='journaling_counts_B2')
B2

,Mood_B
0,2.0
1,2.0
2,2.0
3,2.0
4,2.0
5,2.0
6,3.0
7,3.0
8,3.0
9,2.0


In [289]:
A2 = df1[42:56]
A2 = A2.reset_index().drop('index', axis=1)
A2.rename({'Mood': 'Mood_A', 'journaling_counts':'journaling_counts_A2'}, axis=1, inplace=True)
# A2 = A2.drop(columns='journaling_counts_A2')
A2

,Mood_A
0,2.0
1,2.0
2,2.0
3,2.0
4,2.0
5,2.0
6,2.0
7,2.0
8,2.0
9,3.0


In [290]:
B3 = df1[56:70]
B3 = B3.reset_index().drop('index', axis=1)
B3.rename({'Mood': 'Mood_B', 'journaling_counts':'journaling_counts_B3'}, axis=1, inplace=True)
# B3 = B3.drop(columns='journaling_counts_B3')
B3

,Mood_B
0,2.0
1,2.0
2,2.0
3,2.0
4,2.0
5,2.0
6,2.0
7,2.0
8,2.0
9,2.0


In [291]:
A3 = df1[70:84]
A3 = A3.reset_index().drop('index', axis=1)
A3.rename({'Mood': 'Mood_A', 'journaling_counts':'journaling_counts_A3'}, axis=1, inplace=True)
# A3 = A3.drop(columns='journaling_counts_A3')
A3

,Mood_A
0,2.0
1,2.0
2,1.0
3,2.0
4,2.0
5,2.0
6,2.0
7,2.0
8,2.0
9,2.0


In [292]:
# df2 = pd.concat([B1, A1, B2, A2, B3, A3], axis=1)
# df2

In [293]:
# df3 = df2[['Mood_B', 'Mood_A']]
# df3

In [294]:
df_a = pd.concat([B1, A1], axis=1)
df_a

,Mood_B,Mood_A
0,3.0,2.0
1,3.0,2.0
2,3.0,2.0
3,3.0,2.0
4,2.0,2.0
5,2.0,2.0
6,2.0,2.0
7,3.0,3.0
8,3.0,3.0
9,3.0,2.0


In [295]:
df_b = pd.concat([B2, A2], axis=1)

In [296]:
df_c = pd.concat([B3, A3], axis=1)

In [297]:
df4 = pd.concat({"1": df_a, "2": df_b, "3": df_c}, axis=1)
df4

1             2             3       
   Mood_B Mood_A Mood_B Mood_A Mood_B Mood_A
0     3.0    2.0    2.0    2.0    2.0    2.0
1     3.0    2.0    2.0    2.0    2.0    2.0
2     3.0    2.0    2.0    2.0    2.0    1.0
3     3.0    2.0    2.0    2.0    2.0    2.0
4     2.0    2.0    2.0    2.0    2.0    2.0
5     2.0    2.0    2.0    2.0    2.0    2.0
6     2.0    2.0    3.0    2.0    2.0    2.0
7     3.0    3.0    3.0    2.0    2.0    2.0
8     3.0    3.0    3.0    2.0    2.0    2.0
9     3.0    2.0    2.0    3.0    2.0    2.0
10    3.0    2.0    2.0    2.0    2.0    2.0
11    3.0    2.0    2.0    2.0    2.0    2.0
12    3.0    2.0    2.0    2.0    2.0    2.0
13    3.0    2.0    2.0    2.0    3.0    2.0

In [414]:
df4.to_csv('data/stats/ANOVARM_df.csv', index=None, encoding='utf-8')

In [424]:
describe = df4.describe()
describe

1                     2                     3           
          Mood_B     Mood_A     Mood_B     Mood_A     Mood_B     Mood_A
count  14.000000  14.000000  14.000000  14.000000  14.000000  14.000000
mean    2.785714   2.142857   2.214286   2.071429   2.071429   1.928571
std     0.425815   0.363137   0.425815   0.267261   0.267261   0.267261
min     2.000000   2.000000   2.000000   2.000000   2.000000   1.000000
25%     3.000000   2.000000   2.000000   2.000000   2.000000   2.000000
50%     3.000000   2.000000   2.000000   2.000000   2.000000   2.000000
75%     3.000000   2.000000   2.000000   2.000000   2.000000   2.000000
max     3.000000   3.000000   3.000000   3.000000   3.000000   2.000000

In [425]:
describe.to_csv('data/stats/ANOVARM_df_describe.csv', index=None, encoding='utf-8')

In [383]:
anovarm = pingouin.rm_anova(df4, detailed=True)
anovarm

,Source,SS,DF,MS,F,p-unc,ng2,eps
0,Within,1.583333,2,0.791667,14.529412,0.000058,0.39291,0.951552
1,Error,1.416667,26,0.054487,NaN,NaN,NaN,NaN


In [442]:
anovarm.to_csv('data/stats/ANOVARM_df_results.csv', index=None, encoding='utf-8')

In [404]:
eta_squared = 0.791667 / (0.791667 + 1.416667)
eta_squared

0.3584906087575521

#### Epsilon First

In [182]:
epsilon = round(pingouin.epsilon(df3), 3)
epsilon

0.644

ANOVAs with repeated measures (within-subject factors) are particularly susceptible to the violation of the assumption of sphericity. Sphericity is the condition where the variances of the differences between all combinations of related groups (levels) are equal. Violation of sphericity is when the variances of the differences between all combinations of related groups are not equal. Sphericity can be likened to homogeneity of variances in a between-subjects ANOVA.

Since Journaling has only two levels (0 and 1), the sphericity assumption is necessarily met, and therefore the epsilon adjustement factor is 1.

#### Then Correlation

In [398]:
corr = df3.corr(method='pearson')
# corr = corr['Mood'][1]
corr.mean().mean()

0.2115064943510916

In [184]:
m = 3  # repeated measures
n = 28 # sample size per measurement

In [408]:
# before
print('power: %.4f' % pingouin.power_rm_anova(eta_squared=0.1, m=m, n=n, epsilon=1))

power: 0.9719


In [409]:
# after
print('power: %.4f' % pingouin.power_rm_anova(eta_squared=eta_squared, m=m, n=n, epsilon=epsilon))

power: 1.0000


In [410]:
# generic after
print('power: %.4f' % pingouin.power_rm_anova(eta_squared=0.1, m=m, n=n, epsilon=epsilon))

power: 0.9002


In [411]:
# eta
print('eta-squared: %.4f' % pingouin.power_rm_anova(n=n, m=m, power=0.80, alpha=0.05))

eta-squared: 0.0572


In [188]:
# alpha
print('alpha: %.4f' % pingouin.power_rm_anova(eta_squared=0.1, n=n, m=m, power=0.80, alpha=None))

alpha: 0.0032


In [388]:
# number of phases
print('n: %.4f' % pingouin.power_rm_anova(eta_squared=0.1, corr=.1, power=0.80, n=n, epsilon=1, alpha=0.05))

n: 2.9103


In [389]:
# sample size
print('n: %.4f' % pingouin.power_rm_anova(eta_squared=0.1, corr=.1, power=0.80, m=m, epsilon=1, alpha=0.05))

n: 27.5395


In [189]:
# stats f_oneway functions takes the groups as input and returns ANOVA F and p value
fvalue, pvalue = stats.f_oneway(df2['Mood_B'],
                                df2['Mood_A'],)
print(fvalue, pvalue)

[18.47368421  1.13043478  2.        ] [2.14110744e-04 2.97458397e-01 1.69163028e-01]


In [218]:
# df_melt = pd.melt(df2.reset_index(), id_vars=['index'], value_vars=['Mood_B', 'Mood_A'])
# df_melt = df_melt.drop(columns='index')
# df_melt

In [217]:
# aov = pingouin.rm_anova(dv='value', within='Mood_B', subject='variable', data=df_melt, detailed=True)
# aov

In [219]:
# pivot = df_melt.pivot_table(index=df_melt.index, columns=['variable'], values='value')
# pivot

In [220]:
# https://www.statsmodels.org/dev/generated/statsmodels.formula.api.ols.html#statsmodels.formula.api.ols
# https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLS.html
# https://www.statsmodels.org/dev/examples/notebooks/generated/interactions_anova.html

model = ols('value ~ C(variable)', data=df_melt).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

,sum_sq,df,F,PR(>F)
C(variable),2.011905,1.0,12.177504,0.000781
Residual,13.547619,82.0,NaN,NaN


In [221]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  value   R-squared:                       0.129
Model:                            OLS   Adj. R-squared:                  0.119
Method:                 Least Squares   F-statistic:                     12.18
Date:                Thu, 06 Apr 2023   Prob (F-statistic):           0.000781
Time:                        03:05:26   Log-Likelihood:                -42.557
No. Observations:                  84   AIC:                             89.11
Df Residuals:                      82   BIC:                             93.98
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 2.0476      0.063     32.647      0.000       1.923       2.172
C(variable)[T.Mood_B]     0.3095      0.089      3.490      0.001       0.133       0.486
==============================================================================
Omnibus:                        8.160   Durbin-Watson:                   0.921
Prob(Omnibus):                  0.017   Jarque-Bera (JB):                8.324
Skew:                           0.771   Prob(JB):                       0.0156
Kurtosis:                       3.054   Cond. No.                         2.62
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# ANCOVA

In [305]:
experiment2

,Date,Temperature,Respiration Rate,Vitals Heart Rate,Blood Pressure Systolic,Blood Pressure Diastolic,Mean Arterial Pressure,Step Count,Resting Heart Rate,Number of Floors,Heart Rate Count,Heart Rate Mean,Heart Rate STD,Heart Rate Minimum,Heart Rate 25%,Heart Rate Median,Heart Rate 75%,Heart Rate Maximum,Mood,Pain,Tired,Sick,Anxiousness,Depressed,Suicidal,Self-Harm,Thoughts of Death,Repetitive Thoughts,Thoughts of Worthlessness,Thoughts of Hopelessness,Brain Sludge,Sluggish,Restless,Existential Crisis,Loss of Interest,Self Isolation,Paranoia,Elevated,Irritable/Angry,Loneliness,Libido,Weight,Weird,sleeping_durations_seconds,sleeping_counts,sleeping_durations_seconds_7_day_rolling_mean,internet_durations_seconds,internet_counts,internet_durations_seconds_7_day_rolling_mean,working_durations_seconds,working_counts,working_durations_seconds_7_day_rolling_mean,games_durations_seconds,games_counts,games_durations_seconds_7_day_rolling_mean,family_time_durations_seconds,family_time_counts,family_time_durations_seconds_7_day_rolling_mean,eating_durations_seconds,eating_counts,eating_durations_seconds_7_day_rolling_mean,programming_durations_seconds,programming_counts,programming_durations_seconds_7_day_rolling_mean,grooming_durations_seconds,grooming_counts,grooming_durations_seconds_7_day_rolling_mean,reading_durations_seconds,reading_counts,reading_durations_seconds_7_day_rolling_mean,gym_durations_seconds,gym_counts,gym_durations_seconds_7_day_rolling_mean,toilet_durations_seconds,toilet_counts,toilet_durations_seconds_7_day_rolling_mean,friends_time_durations_seconds,friends_time_counts,friends_time_durations_seconds_7_day_rolling_mean,studying_durations_seconds,studying_counts,studying_durations_seconds_7_day_rolling_mean,movies_tv_durations_seconds,movies_tv_counts,movies_tv_durations_seconds_7_day_rolling_mean,driving_durations_seconds,driving_counts,driving_durations_seconds_7_day_rolling_mean,walking_durations_seconds,walking_counts,walking_durations_seconds_7_day_rolling_mean,writing_durations_seconds,writing_counts,writing_durations_seconds_7_day_rolling_mean,cooking_durations_seconds,cooking_counts,cooking_durations_seconds_7_day_rolling_mean,class_durations_seconds,class_counts,class_durations_seconds_7_day_rolling_mean,cleaning_durations_seconds,cleaning_counts,cleaning_durations_seconds_7_day_rolling_mean,dishes_durations_seconds,dishes_counts,dishes_durations_seconds_7_day_rolling_mean,personal_durations_seconds,personal_counts,personal_durations_seconds_7_day_rolling_mean,dating_durations_seconds,dating_counts,dating_durations_seconds_7_day_rolling_mean,shopping_durations_seconds,shopping_counts,shopping_durations_seconds_7_day_rolling_mean,mycology_durations_seconds,mycology_counts,mycology_durations_seconds_7_day_rolling_mean,errands_and_appointments_durations_seconds,errands_and_appointments_counts,errands_and_appointments_durations_seconds_7_day_rolling_mean,club_social_group_durations_seconds,club_social_group_counts,club_social_group_durations_seconds_7_day_rolling_mean,drinking_durations_seconds,drinking_counts,drinking_durations_seconds_7_day_rolling_mean,gamedev_durations_seconds,gamedev_counts,gamedev_durations_seconds_7_day_rolling_mean,swimming_tanning_durations_seconds,swimming_tanning_counts,swimming_tanning_durations_seconds_7_day_rolling_mean,phone_durations_seconds,phone_counts,phone_durations_seconds_7_day_rolling_mean,keyboard_durations_seconds,keyboard_counts,keyboard_durations_seconds_7_day_rolling_mean,journaling_durations_seconds,journaling_counts,journaling_durations_seconds_7_day_rolling_mean,drawing_durations_seconds,drawing_counts,drawing_durations_seconds_7_day_rolling_mean,traveling__durations_seconds,traveling__counts,traveling__durations_seconds_7_day_rolling_mean,drumming_durations_seconds,drumming_counts,drumming_durations_seconds_7_day_rolling_mean,data_analysis_durations_seconds,data_analysis_counts,data_analysis_durations_seconds_7_day_rolling_mean,concert_event_durations_seconds,co

In [334]:
B1 = experiment2[0:14]
B1 = B1.reset_index().drop('index', axis=1)
B1['index'] = 'Mood_B'
B1 = B1.set_index('index')
# B1.rename({'Mood': 'Mood_B'}, axis=1, inplace=True)
# B1 = B1.drop(columns='journaling_counts_B1')
B1

,Date,Temperature,Respiration Rate,Vitals Heart Rate,Blood Pressure Systolic,Blood Pressure Diastolic,Mean Arterial Pressure,Step Count,Resting Heart Rate,Number of Floors,Heart Rate Count,Heart Rate Mean,Heart Rate STD,Heart Rate Minimum,Heart Rate 25%,Heart Rate Median,Heart Rate 75%,Heart Rate Maximum,Mood,Pain,Tired,Sick,Anxiousness,Depressed,Suicidal,Self-Harm,Thoughts of Death,Repetitive Thoughts,Thoughts of Worthlessness,Thoughts of Hopelessness,Brain Sludge,Sluggish,Restless,Existential Crisis,Loss of Interest,Self Isolation,Paranoia,Elevated,Irritable/Angry,Loneliness,Libido,Weight,Weird,sleeping_durations_seconds,sleeping_counts,sleeping_durations_seconds_7_day_rolling_mean,internet_durations_seconds,internet_counts,internet_durations_seconds_7_day_rolling_mean,working_durations_seconds,working_counts,working_durations_seconds_7_day_rolling_mean,games_durations_seconds,games_counts,games_durations_seconds_7_day_rolling_mean,family_time_durations_seconds,family_time_counts,family_time_durations_seconds_7_day_rolling_mean,eating_durations_seconds,eating_counts,eating_durations_seconds_7_day_rolling_mean,programming_durations_seconds,programming_counts,programming_durations_seconds_7_day_rolling_mean,grooming_durations_seconds,grooming_counts,grooming_durations_seconds_7_day_rolling_mean,reading_durations_seconds,reading_counts,reading_durations_seconds_7_day_rolling_mean,gym_durations_seconds,gym_counts,gym_durations_seconds_7_day_rolling_mean,toilet_durations_seconds,toilet_counts,toilet_durations_seconds_7_day_rolling_mean,friends_time_durations_seconds,friends_time_counts,friends_time_durations_seconds_7_day_rolling_mean,studying_durations_seconds,studying_counts,studying_durations_seconds_7_day_rolling_mean,movies_tv_durations_seconds,movies_tv_counts,movies_tv_durations_seconds_7_day_rolling_mean,driving_durations_seconds,driving_counts,driving_durations_seconds_7_day_rolling_mean,walking_durations_seconds,walking_counts,walking_durations_seconds_7_day_rolling_mean,writing_durations_seconds,writing_counts,writing_durations_seconds_7_day_rolling_mean,cooking_durations_seconds,cooking_counts,cooking_durations_seconds_7_day_rolling_mean,class_durations_seconds,class_counts,class_durations_seconds_7_day_rolling_mean,cleaning_durations_seconds,cleaning_counts,cleaning_durations_seconds_7_day_rolling_mean,dishes_durations_seconds,dishes_counts,dishes_durations_seconds_7_day_rolling_mean,personal_durations_seconds,personal_counts,personal_durations_seconds_7_day_rolling_mean,dating_durations_seconds,dating_counts,dating_durations_seconds_7_day_rolling_mean,shopping_durations_seconds,shopping_counts,shopping_durations_seconds_7_day_rolling_mean,mycology_durations_seconds,mycology_counts,mycology_durations_seconds_7_day_rolling_mean,errands_and_appointments_durations_seconds,errands_and_appointments_counts,errands_and_appointments_durations_seconds_7_day_rolling_mean,club_social_group_durations_seconds,club_social_group_counts,club_social_group_durations_seconds_7_day_rolling_mean,drinking_durations_seconds,drinking_counts,drinking_durations_seconds_7_day_rolling_mean,gamedev_durations_seconds,gamedev_counts,gamedev_durations_seconds_7_day_rolling_mean,swimming_tanning_durations_seconds,swimming_tanning_counts,swimming_tanning_durations_seconds_7_day_rolling_mean,phone_durations_seconds,phone_counts,phone_durations_seconds_7_day_rolling_mean,keyboard_durations_seconds,keyboard_counts,keyboard_durations_seconds_7_day_rolling_mean,journaling_durations_seconds,journaling_counts,journaling_durations_seconds_7_day_rolling_mean,drawing_durations_seconds,drawing_counts,drawing_durations_seconds_7_day_rolling_mean,traveling__durations_seconds,traveling__counts,traveling__durations_seconds_7_day_rolling_mean,drumming_durations_seconds,drumming_counts,drumming_durations_seconds_7_day_rolling_mean,data_analysis_durations_seconds,data_analysis_counts,data_analysis_durations_seconds_7_day_rolling_mean,concert_event_durations_seconds,co

In [335]:
A1 = experiment2[14:28]
A1 = A1.reset_index().drop('index', axis=1)
A1['index'] = 'Mood_A'
A1 = A1.set_index('index')
# A1.rename({'Mood': 'Mood_A'}, axis=1, inplace=True)
# A1 = A1.drop(columns='journaling_counts_A1')
A1

,Date,Temperature,Respiration Rate,Vitals Heart Rate,Blood Pressure Systolic,Blood Pressure Diastolic,Mean Arterial Pressure,Step Count,Resting Heart Rate,Number of Floors,Heart Rate Count,Heart Rate Mean,Heart Rate STD,Heart Rate Minimum,Heart Rate 25%,Heart Rate Median,Heart Rate 75%,Heart Rate Maximum,Mood,Pain,Tired,Sick,Anxiousness,Depressed,Suicidal,Self-Harm,Thoughts of Death,Repetitive Thoughts,Thoughts of Worthlessness,Thoughts of Hopelessness,Brain Sludge,Sluggish,Restless,Existential Crisis,Loss of Interest,Self Isolation,Paranoia,Elevated,Irritable/Angry,Loneliness,Libido,Weight,Weird,sleeping_durations_seconds,sleeping_counts,sleeping_durations_seconds_7_day_rolling_mean,internet_durations_seconds,internet_counts,internet_durations_seconds_7_day_rolling_mean,working_durations_seconds,working_counts,working_durations_seconds_7_day_rolling_mean,games_durations_seconds,games_counts,games_durations_seconds_7_day_rolling_mean,family_time_durations_seconds,family_time_counts,family_time_durations_seconds_7_day_rolling_mean,eating_durations_seconds,eating_counts,eating_durations_seconds_7_day_rolling_mean,programming_durations_seconds,programming_counts,programming_durations_seconds_7_day_rolling_mean,grooming_durations_seconds,grooming_counts,grooming_durations_seconds_7_day_rolling_mean,reading_durations_seconds,reading_counts,reading_durations_seconds_7_day_rolling_mean,gym_durations_seconds,gym_counts,gym_durations_seconds_7_day_rolling_mean,toilet_durations_seconds,toilet_counts,toilet_durations_seconds_7_day_rolling_mean,friends_time_durations_seconds,friends_time_counts,friends_time_durations_seconds_7_day_rolling_mean,studying_durations_seconds,studying_counts,studying_durations_seconds_7_day_rolling_mean,movies_tv_durations_seconds,movies_tv_counts,movies_tv_durations_seconds_7_day_rolling_mean,driving_durations_seconds,driving_counts,driving_durations_seconds_7_day_rolling_mean,walking_durations_seconds,walking_counts,walking_durations_seconds_7_day_rolling_mean,writing_durations_seconds,writing_counts,writing_durations_seconds_7_day_rolling_mean,cooking_durations_seconds,cooking_counts,cooking_durations_seconds_7_day_rolling_mean,class_durations_seconds,class_counts,class_durations_seconds_7_day_rolling_mean,cleaning_durations_seconds,cleaning_counts,cleaning_durations_seconds_7_day_rolling_mean,dishes_durations_seconds,dishes_counts,dishes_durations_seconds_7_day_rolling_mean,personal_durations_seconds,personal_counts,personal_durations_seconds_7_day_rolling_mean,dating_durations_seconds,dating_counts,dating_durations_seconds_7_day_rolling_mean,shopping_durations_seconds,shopping_counts,shopping_durations_seconds_7_day_rolling_mean,mycology_durations_seconds,mycology_counts,mycology_durations_seconds_7_day_rolling_mean,errands_and_appointments_durations_seconds,errands_and_appointments_counts,errands_and_appointments_durations_seconds_7_day_rolling_mean,club_social_group_durations_seconds,club_social_group_counts,club_social_group_durations_seconds_7_day_rolling_mean,drinking_durations_seconds,drinking_counts,drinking_durations_seconds_7_day_rolling_mean,gamedev_durations_seconds,gamedev_counts,gamedev_durations_seconds_7_day_rolling_mean,swimming_tanning_durations_seconds,swimming_tanning_counts,swimming_tanning_durations_seconds_7_day_rolling_mean,phone_durations_seconds,phone_counts,phone_durations_seconds_7_day_rolling_mean,keyboard_durations_seconds,keyboard_counts,keyboard_durations_seconds_7_day_rolling_mean,journaling_durations_seconds,journaling_counts,journaling_durations_seconds_7_day_rolling_mean,drawing_durations_seconds,drawing_counts,drawing_durations_seconds_7_day_rolling_mean,traveling__durations_seconds,traveling__counts,traveling__durations_seconds_7_day_rolling_mean,drumming_durations_seconds,drumming_counts,drumming_durations_seconds_7_day_rolling_mean,data_analysis_durations_seconds,data_analysis_counts,data_analysis_durations_seconds_7_day_rolling_mean,concert_event_durations_seconds,co

In [336]:
B2 = experiment2[28:42]
B2 = B2.reset_index().drop('index', axis=1)
B2['index'] = 'Mood_B'
B2 = B2.set_index('index')
# B2.rename({'Mood': 'Mood_B'}, axis=1, inplace=True)
# B2 = B2.drop(columns='journaling_counts_B2')
B2

,Date,Temperature,Respiration Rate,Vitals Heart Rate,Blood Pressure Systolic,Blood Pressure Diastolic,Mean Arterial Pressure,Step Count,Resting Heart Rate,Number of Floors,Heart Rate Count,Heart Rate Mean,Heart Rate STD,Heart Rate Minimum,Heart Rate 25%,Heart Rate Median,Heart Rate 75%,Heart Rate Maximum,Mood,Pain,Tired,Sick,Anxiousness,Depressed,Suicidal,Self-Harm,Thoughts of Death,Repetitive Thoughts,Thoughts of Worthlessness,Thoughts of Hopelessness,Brain Sludge,Sluggish,Restless,Existential Crisis,Loss of Interest,Self Isolation,Paranoia,Elevated,Irritable/Angry,Loneliness,Libido,Weight,Weird,sleeping_durations_seconds,sleeping_counts,sleeping_durations_seconds_7_day_rolling_mean,internet_durations_seconds,internet_counts,internet_durations_seconds_7_day_rolling_mean,working_durations_seconds,working_counts,working_durations_seconds_7_day_rolling_mean,games_durations_seconds,games_counts,games_durations_seconds_7_day_rolling_mean,family_time_durations_seconds,family_time_counts,family_time_durations_seconds_7_day_rolling_mean,eating_durations_seconds,eating_counts,eating_durations_seconds_7_day_rolling_mean,programming_durations_seconds,programming_counts,programming_durations_seconds_7_day_rolling_mean,grooming_durations_seconds,grooming_counts,grooming_durations_seconds_7_day_rolling_mean,reading_durations_seconds,reading_counts,reading_durations_seconds_7_day_rolling_mean,gym_durations_seconds,gym_counts,gym_durations_seconds_7_day_rolling_mean,toilet_durations_seconds,toilet_counts,toilet_durations_seconds_7_day_rolling_mean,friends_time_durations_seconds,friends_time_counts,friends_time_durations_seconds_7_day_rolling_mean,studying_durations_seconds,studying_counts,studying_durations_seconds_7_day_rolling_mean,movies_tv_durations_seconds,movies_tv_counts,movies_tv_durations_seconds_7_day_rolling_mean,driving_durations_seconds,driving_counts,driving_durations_seconds_7_day_rolling_mean,walking_durations_seconds,walking_counts,walking_durations_seconds_7_day_rolling_mean,writing_durations_seconds,writing_counts,writing_durations_seconds_7_day_rolling_mean,cooking_durations_seconds,cooking_counts,cooking_durations_seconds_7_day_rolling_mean,class_durations_seconds,class_counts,class_durations_seconds_7_day_rolling_mean,cleaning_durations_seconds,cleaning_counts,cleaning_durations_seconds_7_day_rolling_mean,dishes_durations_seconds,dishes_counts,dishes_durations_seconds_7_day_rolling_mean,personal_durations_seconds,personal_counts,personal_durations_seconds_7_day_rolling_mean,dating_durations_seconds,dating_counts,dating_durations_seconds_7_day_rolling_mean,shopping_durations_seconds,shopping_counts,shopping_durations_seconds_7_day_rolling_mean,mycology_durations_seconds,mycology_counts,mycology_durations_seconds_7_day_rolling_mean,errands_and_appointments_durations_seconds,errands_and_appointments_counts,errands_and_appointments_durations_seconds_7_day_rolling_mean,club_social_group_durations_seconds,club_social_group_counts,club_social_group_durations_seconds_7_day_rolling_mean,drinking_durations_seconds,drinking_counts,drinking_durations_seconds_7_day_rolling_mean,gamedev_durations_seconds,gamedev_counts,gamedev_durations_seconds_7_day_rolling_mean,swimming_tanning_durations_seconds,swimming_tanning_counts,swimming_tanning_durations_seconds_7_day_rolling_mean,phone_durations_seconds,phone_counts,phone_durations_seconds_7_day_rolling_mean,keyboard_durations_seconds,keyboard_counts,keyboard_durations_seconds_7_day_rolling_mean,journaling_durations_seconds,journaling_counts,journaling_durations_seconds_7_day_rolling_mean,drawing_durations_seconds,drawing_counts,drawing_durations_seconds_7_day_rolling_mean,traveling__durations_seconds,traveling__counts,traveling__durations_seconds_7_day_rolling_mean,drumming_durations_seconds,drumming_counts,drumming_durations_seconds_7_day_rolling_mean,data_analysis_durations_seconds,data_analysis_counts,data_analysis_durations_seconds_7_day_rolling_mean,concert_event_durations_seconds,co

In [337]:
A2 = experiment2[42:56]
A2 = A2.reset_index().drop('index', axis=1)
A2['index'] = 'Mood_A'
A2 = A2.set_index('index')
# A2.rename({'Mood': 'Mood_A'}, axis=1, inplace=True)
# A2 = A2.drop(columns='journaling_counts_A2')
A2

,Date,Temperature,Respiration Rate,Vitals Heart Rate,Blood Pressure Systolic,Blood Pressure Diastolic,Mean Arterial Pressure,Step Count,Resting Heart Rate,Number of Floors,Heart Rate Count,Heart Rate Mean,Heart Rate STD,Heart Rate Minimum,Heart Rate 25%,Heart Rate Median,Heart Rate 75%,Heart Rate Maximum,Mood,Pain,Tired,Sick,Anxiousness,Depressed,Suicidal,Self-Harm,Thoughts of Death,Repetitive Thoughts,Thoughts of Worthlessness,Thoughts of Hopelessness,Brain Sludge,Sluggish,Restless,Existential Crisis,Loss of Interest,Self Isolation,Paranoia,Elevated,Irritable/Angry,Loneliness,Libido,Weight,Weird,sleeping_durations_seconds,sleeping_counts,sleeping_durations_seconds_7_day_rolling_mean,internet_durations_seconds,internet_counts,internet_durations_seconds_7_day_rolling_mean,working_durations_seconds,working_counts,working_durations_seconds_7_day_rolling_mean,games_durations_seconds,games_counts,games_durations_seconds_7_day_rolling_mean,family_time_durations_seconds,family_time_counts,family_time_durations_seconds_7_day_rolling_mean,eating_durations_seconds,eating_counts,eating_durations_seconds_7_day_rolling_mean,programming_durations_seconds,programming_counts,programming_durations_seconds_7_day_rolling_mean,grooming_durations_seconds,grooming_counts,grooming_durations_seconds_7_day_rolling_mean,reading_durations_seconds,reading_counts,reading_durations_seconds_7_day_rolling_mean,gym_durations_seconds,gym_counts,gym_durations_seconds_7_day_rolling_mean,toilet_durations_seconds,toilet_counts,toilet_durations_seconds_7_day_rolling_mean,friends_time_durations_seconds,friends_time_counts,friends_time_durations_seconds_7_day_rolling_mean,studying_durations_seconds,studying_counts,studying_durations_seconds_7_day_rolling_mean,movies_tv_durations_seconds,movies_tv_counts,movies_tv_durations_seconds_7_day_rolling_mean,driving_durations_seconds,driving_counts,driving_durations_seconds_7_day_rolling_mean,walking_durations_seconds,walking_counts,walking_durations_seconds_7_day_rolling_mean,writing_durations_seconds,writing_counts,writing_durations_seconds_7_day_rolling_mean,cooking_durations_seconds,cooking_counts,cooking_durations_seconds_7_day_rolling_mean,class_durations_seconds,class_counts,class_durations_seconds_7_day_rolling_mean,cleaning_durations_seconds,cleaning_counts,cleaning_durations_seconds_7_day_rolling_mean,dishes_durations_seconds,dishes_counts,dishes_durations_seconds_7_day_rolling_mean,personal_durations_seconds,personal_counts,personal_durations_seconds_7_day_rolling_mean,dating_durations_seconds,dating_counts,dating_durations_seconds_7_day_rolling_mean,shopping_durations_seconds,shopping_counts,shopping_durations_seconds_7_day_rolling_mean,mycology_durations_seconds,mycology_counts,mycology_durations_seconds_7_day_rolling_mean,errands_and_appointments_durations_seconds,errands_and_appointments_counts,errands_and_appointments_durations_seconds_7_day_rolling_mean,club_social_group_durations_seconds,club_social_group_counts,club_social_group_durations_seconds_7_day_rolling_mean,drinking_durations_seconds,drinking_counts,drinking_durations_seconds_7_day_rolling_mean,gamedev_durations_seconds,gamedev_counts,gamedev_durations_seconds_7_day_rolling_mean,swimming_tanning_durations_seconds,swimming_tanning_counts,swimming_tanning_durations_seconds_7_day_rolling_mean,phone_durations_seconds,phone_counts,phone_durations_seconds_7_day_rolling_mean,keyboard_durations_seconds,keyboard_counts,keyboard_durations_seconds_7_day_rolling_mean,journaling_durations_seconds,journaling_counts,journaling_durations_seconds_7_day_rolling_mean,drawing_durations_seconds,drawing_counts,drawing_durations_seconds_7_day_rolling_mean,traveling__durations_seconds,traveling__counts,traveling__durations_seconds_7_day_rolling_mean,drumming_durations_seconds,drumming_counts,drumming_durations_seconds_7_day_rolling_mean,data_analysis_durations_seconds,data_analysis_counts,data_analysis_durations_seconds_7_day_rolling_mean,concert_event_durations_seconds,co

In [338]:
B3 = experiment2[56:70]
B3 = B3.reset_index().drop('index', axis=1)
B3['index'] = 'Mood_B'
B3 = B3.set_index('index')
# B3.rename({'Mood': 'Mood_B'}, axis=1, inplace=True)
# B3 = B3.drop(columns='journaling_counts_B3')
B3

,Date,Temperature,Respiration Rate,Vitals Heart Rate,Blood Pressure Systolic,Blood Pressure Diastolic,Mean Arterial Pressure,Step Count,Resting Heart Rate,Number of Floors,Heart Rate Count,Heart Rate Mean,Heart Rate STD,Heart Rate Minimum,Heart Rate 25%,Heart Rate Median,Heart Rate 75%,Heart Rate Maximum,Mood,Pain,Tired,Sick,Anxiousness,Depressed,Suicidal,Self-Harm,Thoughts of Death,Repetitive Thoughts,Thoughts of Worthlessness,Thoughts of Hopelessness,Brain Sludge,Sluggish,Restless,Existential Crisis,Loss of Interest,Self Isolation,Paranoia,Elevated,Irritable/Angry,Loneliness,Libido,Weight,Weird,sleeping_durations_seconds,sleeping_counts,sleeping_durations_seconds_7_day_rolling_mean,internet_durations_seconds,internet_counts,internet_durations_seconds_7_day_rolling_mean,working_durations_seconds,working_counts,working_durations_seconds_7_day_rolling_mean,games_durations_seconds,games_counts,games_durations_seconds_7_day_rolling_mean,family_time_durations_seconds,family_time_counts,family_time_durations_seconds_7_day_rolling_mean,eating_durations_seconds,eating_counts,eating_durations_seconds_7_day_rolling_mean,programming_durations_seconds,programming_counts,programming_durations_seconds_7_day_rolling_mean,grooming_durations_seconds,grooming_counts,grooming_durations_seconds_7_day_rolling_mean,reading_durations_seconds,reading_counts,reading_durations_seconds_7_day_rolling_mean,gym_durations_seconds,gym_counts,gym_durations_seconds_7_day_rolling_mean,toilet_durations_seconds,toilet_counts,toilet_durations_seconds_7_day_rolling_mean,friends_time_durations_seconds,friends_time_counts,friends_time_durations_seconds_7_day_rolling_mean,studying_durations_seconds,studying_counts,studying_durations_seconds_7_day_rolling_mean,movies_tv_durations_seconds,movies_tv_counts,movies_tv_durations_seconds_7_day_rolling_mean,driving_durations_seconds,driving_counts,driving_durations_seconds_7_day_rolling_mean,walking_durations_seconds,walking_counts,walking_durations_seconds_7_day_rolling_mean,writing_durations_seconds,writing_counts,writing_durations_seconds_7_day_rolling_mean,cooking_durations_seconds,cooking_counts,cooking_durations_seconds_7_day_rolling_mean,class_durations_seconds,class_counts,class_durations_seconds_7_day_rolling_mean,cleaning_durations_seconds,cleaning_counts,cleaning_durations_seconds_7_day_rolling_mean,dishes_durations_seconds,dishes_counts,dishes_durations_seconds_7_day_rolling_mean,personal_durations_seconds,personal_counts,personal_durations_seconds_7_day_rolling_mean,dating_durations_seconds,dating_counts,dating_durations_seconds_7_day_rolling_mean,shopping_durations_seconds,shopping_counts,shopping_durations_seconds_7_day_rolling_mean,mycology_durations_seconds,mycology_counts,mycology_durations_seconds_7_day_rolling_mean,errands_and_appointments_durations_seconds,errands_and_appointments_counts,errands_and_appointments_durations_seconds_7_day_rolling_mean,club_social_group_durations_seconds,club_social_group_counts,club_social_group_durations_seconds_7_day_rolling_mean,drinking_durations_seconds,drinking_counts,drinking_durations_seconds_7_day_rolling_mean,gamedev_durations_seconds,gamedev_counts,gamedev_durations_seconds_7_day_rolling_mean,swimming_tanning_durations_seconds,swimming_tanning_counts,swimming_tanning_durations_seconds_7_day_rolling_mean,phone_durations_seconds,phone_counts,phone_durations_seconds_7_day_rolling_mean,keyboard_durations_seconds,keyboard_counts,keyboard_durations_seconds_7_day_rolling_mean,journaling_durations_seconds,journaling_counts,journaling_durations_seconds_7_day_rolling_mean,drawing_durations_seconds,drawing_counts,drawing_durations_seconds_7_day_rolling_mean,traveling__durations_seconds,traveling__counts,traveling__durations_seconds_7_day_rolling_mean,drumming_durations_seconds,drumming_counts,drumming_durations_seconds_7_day_rolling_mean,data_analysis_durations_seconds,data_analysis_counts,data_analysis_durations_seconds_7_day_rolling_mean,concert_event_durations_seconds,co

In [339]:
A3 = experiment2[70:84]
A3 = A3.reset_index().drop('index', axis=1)
A3['index'] = 'Mood_B'
A3 = A3.set_index('index')
# A3.rename({'Mood': 'Mood_A'}, axis=1, inplace=True)
# A3 = A3.drop(columns='journaling_counts_A3')
A3

,Date,Temperature,Respiration Rate,Vitals Heart Rate,Blood Pressure Systolic,Blood Pressure Diastolic,Mean Arterial Pressure,Step Count,Resting Heart Rate,Number of Floors,Heart Rate Count,Heart Rate Mean,Heart Rate STD,Heart Rate Minimum,Heart Rate 25%,Heart Rate Median,Heart Rate 75%,Heart Rate Maximum,Mood,Pain,Tired,Sick,Anxiousness,Depressed,Suicidal,Self-Harm,Thoughts of Death,Repetitive Thoughts,Thoughts of Worthlessness,Thoughts of Hopelessness,Brain Sludge,Sluggish,Restless,Existential Crisis,Loss of Interest,Self Isolation,Paranoia,Elevated,Irritable/Angry,Loneliness,Libido,Weight,Weird,sleeping_durations_seconds,sleeping_counts,sleeping_durations_seconds_7_day_rolling_mean,internet_durations_seconds,internet_counts,internet_durations_seconds_7_day_rolling_mean,working_durations_seconds,working_counts,working_durations_seconds_7_day_rolling_mean,games_durations_seconds,games_counts,games_durations_seconds_7_day_rolling_mean,family_time_durations_seconds,family_time_counts,family_time_durations_seconds_7_day_rolling_mean,eating_durations_seconds,eating_counts,eating_durations_seconds_7_day_rolling_mean,programming_durations_seconds,programming_counts,programming_durations_seconds_7_day_rolling_mean,grooming_durations_seconds,grooming_counts,grooming_durations_seconds_7_day_rolling_mean,reading_durations_seconds,reading_counts,reading_durations_seconds_7_day_rolling_mean,gym_durations_seconds,gym_counts,gym_durations_seconds_7_day_rolling_mean,toilet_durations_seconds,toilet_counts,toilet_durations_seconds_7_day_rolling_mean,friends_time_durations_seconds,friends_time_counts,friends_time_durations_seconds_7_day_rolling_mean,studying_durations_seconds,studying_counts,studying_durations_seconds_7_day_rolling_mean,movies_tv_durations_seconds,movies_tv_counts,movies_tv_durations_seconds_7_day_rolling_mean,driving_durations_seconds,driving_counts,driving_durations_seconds_7_day_rolling_mean,walking_durations_seconds,walking_counts,walking_durations_seconds_7_day_rolling_mean,writing_durations_seconds,writing_counts,writing_durations_seconds_7_day_rolling_mean,cooking_durations_seconds,cooking_counts,cooking_durations_seconds_7_day_rolling_mean,class_durations_seconds,class_counts,class_durations_seconds_7_day_rolling_mean,cleaning_durations_seconds,cleaning_counts,cleaning_durations_seconds_7_day_rolling_mean,dishes_durations_seconds,dishes_counts,dishes_durations_seconds_7_day_rolling_mean,personal_durations_seconds,personal_counts,personal_durations_seconds_7_day_rolling_mean,dating_durations_seconds,dating_counts,dating_durations_seconds_7_day_rolling_mean,shopping_durations_seconds,shopping_counts,shopping_durations_seconds_7_day_rolling_mean,mycology_durations_seconds,mycology_counts,mycology_durations_seconds_7_day_rolling_mean,errands_and_appointments_durations_seconds,errands_and_appointments_counts,errands_and_appointments_durations_seconds_7_day_rolling_mean,club_social_group_durations_seconds,club_social_group_counts,club_social_group_durations_seconds_7_day_rolling_mean,drinking_durations_seconds,drinking_counts,drinking_durations_seconds_7_day_rolling_mean,gamedev_durations_seconds,gamedev_counts,gamedev_durations_seconds_7_day_rolling_mean,swimming_tanning_durations_seconds,swimming_tanning_counts,swimming_tanning_durations_seconds_7_day_rolling_mean,phone_durations_seconds,phone_counts,phone_durations_seconds_7_day_rolling_mean,keyboard_durations_seconds,keyboard_counts,keyboard_durations_seconds_7_day_rolling_mean,journaling_durations_seconds,journaling_counts,journaling_durations_seconds_7_day_rolling_mean,drawing_durations_seconds,drawing_counts,drawing_durations_seconds_7_day_rolling_mean,traveling__durations_seconds,traveling__counts,traveling__durations_seconds_7_day_rolling_mean,drumming_durations_seconds,drumming_counts,drumming_durations_seconds_7_day_rolling_mean,data_analysis_durations_seconds,data_analysis_counts,data_analysis_durations_seconds_7_day_rolling_mean,concert_event_durations_seconds,co

In [340]:
df5 = pd.concat([B1, A1, B2, A2, B3, A3])
df5

,Date,Temperature,Respiration Rate,Vitals Heart Rate,Blood Pressure Systolic,Blood Pressure Diastolic,Mean Arterial Pressure,Step Count,Resting Heart Rate,Number of Floors,Heart Rate Count,Heart Rate Mean,Heart Rate STD,Heart Rate Minimum,Heart Rate 25%,Heart Rate Median,Heart Rate 75%,Heart Rate Maximum,Mood,Pain,Tired,Sick,Anxiousness,Depressed,Suicidal,Self-Harm,Thoughts of Death,Repetitive Thoughts,Thoughts of Worthlessness,Thoughts of Hopelessness,Brain Sludge,Sluggish,Restless,Existential Crisis,Loss of Interest,Self Isolation,Paranoia,Elevated,Irritable/Angry,Loneliness,Libido,Weight,Weird,sleeping_durations_seconds,sleeping_counts,sleeping_durations_seconds_7_day_rolling_mean,internet_durations_seconds,internet_counts,internet_durations_seconds_7_day_rolling_mean,working_durations_seconds,working_counts,working_durations_seconds_7_day_rolling_mean,games_durations_seconds,games_counts,games_durations_seconds_7_day_rolling_mean,family_time_durations_seconds,family_time_counts,family_time_durations_seconds_7_day_rolling_mean,eating_durations_seconds,eating_counts,eating_durations_seconds_7_day_rolling_mean,programming_durations_seconds,programming_counts,programming_durations_seconds_7_day_rolling_mean,grooming_durations_seconds,grooming_counts,grooming_durations_seconds_7_day_rolling_mean,reading_durations_seconds,reading_counts,reading_durations_seconds_7_day_rolling_mean,gym_durations_seconds,gym_counts,gym_durations_seconds_7_day_rolling_mean,toilet_durations_seconds,toilet_counts,toilet_durations_seconds_7_day_rolling_mean,friends_time_durations_seconds,friends_time_counts,friends_time_durations_seconds_7_day_rolling_mean,studying_durations_seconds,studying_counts,studying_durations_seconds_7_day_rolling_mean,movies_tv_durations_seconds,movies_tv_counts,movies_tv_durations_seconds_7_day_rolling_mean,driving_durations_seconds,driving_counts,driving_durations_seconds_7_day_rolling_mean,walking_durations_seconds,walking_counts,walking_durations_seconds_7_day_rolling_mean,writing_durations_seconds,writing_counts,writing_durations_seconds_7_day_rolling_mean,cooking_durations_seconds,cooking_counts,cooking_durations_seconds_7_day_rolling_mean,class_durations_seconds,class_counts,class_durations_seconds_7_day_rolling_mean,cleaning_durations_seconds,cleaning_counts,cleaning_durations_seconds_7_day_rolling_mean,dishes_durations_seconds,dishes_counts,dishes_durations_seconds_7_day_rolling_mean,personal_durations_seconds,personal_counts,personal_durations_seconds_7_day_rolling_mean,dating_durations_seconds,dating_counts,dating_durations_seconds_7_day_rolling_mean,shopping_durations_seconds,shopping_counts,shopping_durations_seconds_7_day_rolling_mean,mycology_durations_seconds,mycology_counts,mycology_durations_seconds_7_day_rolling_mean,errands_and_appointments_durations_seconds,errands_and_appointments_counts,errands_and_appointments_durations_seconds_7_day_rolling_mean,club_social_group_durations_seconds,club_social_group_counts,club_social_group_durations_seconds_7_day_rolling_mean,drinking_durations_seconds,drinking_counts,drinking_durations_seconds_7_day_rolling_mean,gamedev_durations_seconds,gamedev_counts,gamedev_durations_seconds_7_day_rolling_mean,swimming_tanning_durations_seconds,swimming_tanning_counts,swimming_tanning_durations_seconds_7_day_rolling_mean,phone_durations_seconds,phone_counts,phone_durations_seconds_7_day_rolling_mean,keyboard_durations_seconds,keyboard_counts,keyboard_durations_seconds_7_day_rolling_mean,journaling_durations_seconds,journaling_counts,journaling_durations_seconds_7_day_rolling_mean,drawing_durations_seconds,drawing_counts,drawing_durations_seconds_7_day_rolling_mean,traveling__durations_seconds,traveling__counts,traveling__durations_seconds_7_day_rolling_mean,drumming_durations_seconds,drumming_counts,drumming_durations_seconds_7_day_rolling_mean,data_analysis_durations_seconds,data_analysis_counts,data_analysis_durations_seconds_7_day_rolling_mean,concert_event_durations_seconds,co

In [377]:
df5 = df5.rename(columns={'index': 'Phases'})

In [441]:
# # example
# data = pd.DataFrame({'methodology': np.repeat(['A', 'B', 'C'], 4),
#                      'current_grade': [67, 88, 75, 85,
#                                        92, 77, 74, 88,
#                                        91, 88, 82, 80],
#                      'test_score': [77, 89, 74, 69,
#                                     88, 93, 94, 90,
#                                     85, 81, 83, 79]})
 
# data

In [374]:
# example
# pingouin.ancova(data=data, dv='test_score', covar='current_grade', between='methodology')

,Source,SS,DF,F,p-unc,np2
0,methodology,387.510992,2,6.005703,0.025542,0.600228
1,current_grade,1.404669,1,0.043540,0.839927,0.005413
2,Residual,258.095331,8,NaN,NaN,NaN


In [352]:
# df5 = df5.reset_index()
# df5 = df5.drop(columns='Date')
df5 = df5.drop(columns='Date_shifted')

,index,Temperature,Respiration Rate,Vitals Heart Rate,Blood Pressure Systolic,Blood Pressure Diastolic,Mean Arterial Pressure,Step Count,Resting Heart Rate,Number of Floors,Heart Rate Count,Heart Rate Mean,Heart Rate STD,Heart Rate Minimum,Heart Rate 25%,Heart Rate Median,Heart Rate 75%,Heart Rate Maximum,Mood,Pain,Tired,Sick,Anxiousness,Depressed,Suicidal,Self-Harm,Thoughts of Death,Repetitive Thoughts,Thoughts of Worthlessness,Thoughts of Hopelessness,Brain Sludge,Sluggish,Restless,Existential Crisis,Loss of Interest,Self Isolation,Paranoia,Elevated,Irritable/Angry,Loneliness,Libido,Weight,Weird,sleeping_durations_seconds,sleeping_counts,sleeping_durations_seconds_7_day_rolling_mean,internet_durations_seconds,internet_counts,internet_durations_seconds_7_day_rolling_mean,working_durations_seconds,working_counts,working_durations_seconds_7_day_rolling_mean,games_durations_seconds,games_counts,games_durations_seconds_7_day_rolling_mean,family_time_durations_seconds,family_time_counts,family_time_durations_seconds_7_day_rolling_mean,eating_durations_seconds,eating_counts,eating_durations_seconds_7_day_rolling_mean,programming_durations_seconds,programming_counts,programming_durations_seconds_7_day_rolling_mean,grooming_durations_seconds,grooming_counts,grooming_durations_seconds_7_day_rolling_mean,reading_durations_seconds,reading_counts,reading_durations_seconds_7_day_rolling_mean,gym_durations_seconds,gym_counts,gym_durations_seconds_7_day_rolling_mean,toilet_durations_seconds,toilet_counts,toilet_durations_seconds_7_day_rolling_mean,friends_time_durations_seconds,friends_time_counts,friends_time_durations_seconds_7_day_rolling_mean,studying_durations_seconds,studying_counts,studying_durations_seconds_7_day_rolling_mean,movies_tv_durations_seconds,movies_tv_counts,movies_tv_durations_seconds_7_day_rolling_mean,driving_durations_seconds,driving_counts,driving_durations_seconds_7_day_rolling_mean,walking_durations_seconds,walking_counts,walking_durations_seconds_7_day_rolling_mean,writing_durations_seconds,writing_counts,writing_durations_seconds_7_day_rolling_mean,cooking_durations_seconds,cooking_counts,cooking_durations_seconds_7_day_rolling_mean,class_durations_seconds,class_counts,class_durations_seconds_7_day_rolling_mean,cleaning_durations_seconds,cleaning_counts,cleaning_durations_seconds_7_day_rolling_mean,dishes_durations_seconds,dishes_counts,dishes_durations_seconds_7_day_rolling_mean,personal_durations_seconds,personal_counts,personal_durations_seconds_7_day_rolling_mean,dating_durations_seconds,dating_counts,dating_durations_seconds_7_day_rolling_mean,shopping_durations_seconds,shopping_counts,shopping_durations_seconds_7_day_rolling_mean,mycology_durations_seconds,mycology_counts,mycology_durations_seconds_7_day_rolling_mean,errands_and_appointments_durations_seconds,errands_and_appointments_counts,errands_and_appointments_durations_seconds_7_day_rolling_mean,club_social_group_durations_seconds,club_social_group_counts,club_social_group_durations_seconds_7_day_rolling_mean,drinking_durations_seconds,drinking_counts,drinking_durations_seconds_7_day_rolling_mean,gamedev_durations_seconds,gamedev_counts,gamedev_durations_seconds_7_day_rolling_mean,swimming_tanning_durations_seconds,swimming_tanning_counts,swimming_tanning_durations_seconds_7_day_rolling_mean,phone_durations_seconds,phone_counts,phone_durations_seconds_7_day_rolling_mean,keyboard_durations_seconds,keyboard_counts,keyboard_durations_seconds_7_day_rolling_mean,journaling_durations_seconds,journaling_counts,journaling_durations_seconds_7_day_rolling_mean,drawing_durations_seconds,drawing_counts,drawing_durations_seconds_7_day_rolling_mean,traveling__durations_seconds,traveling__counts,traveling__durations_seconds_7_day_rolling_mean,drumming_durations_seconds,drumming_counts,drumming_durations_seconds_7_day_rolling_mean,data_analysis_durations_seconds,data_analysis_counts,data_analysis_durations_seconds_7_day_rolling_mean,concert_event_durations_seconds,c

In [378]:
keep = ['Mood', 'Phases']
columns = [col for col in df5.columns if col not in keep]

In [379]:
columns = ['Pain', 
           'Tired',
           'Sick',
           'Anxiousness', 
           'Depressed', 
           'Suicidal',
           'Self-Harm',
           'Thoughts of Death', 
           'Repetitive Thoughts',
           'Thoughts of Worthlessness', 
           'Thoughts of Hopelessness',
           'Brain Sludge', 
           'Sluggish', 
           'Restless', 
           'Existential Crisis', 
           'Weird',
           'Loss of Interest', 
           'Self Isolation', 
           'Paranoia',
           'Elevated', 
           'Irritable/Angry',
           'Loneliness', 
           'Libido', 
           'Weight']

In [380]:
vitals = ['Step Count',
          'Resting Heart Rate',
          'Number of Floors',
          'Heart Rate Count',
          'Heart Rate Mean',
          'Heart Rate STD',
          'Heart Rate Minimum',
          'Heart Rate 25%',
          'Heart Rate Median',
          'Heart Rate 75%',
          'Heart Rate Maximum',]

df_counts = [i for i in df5.columns if i.endswith('counts')]
df_durations_seconds = [i for i in df5.columns if i.endswith('seconds')]
df_durations = [i for i in df5.columns if i.endswith('durations')]
df_durations_seconds_7_day_rolling_mean = [i for i in df5.columns if i.endswith('mean')]
df_shifted = [i for i in df5.columns if i.endswith('shifted')]

In [449]:
ancova = pingouin.ancova(data=df5, dv='Mood', 
                         covar=#'journaling_durations_seconds', 
                         'journaling_counts', 
                         #'journaling_durations_seconds_7_day_rolling_mean'], 
                         between='Phases')
ancova

,Source,SS,DF,F,p-unc,np2
0,Phases,0.297619,1,1.819407,0.181143,0.021968
1,journaling_counts,1.928571,1,11.789757,0.000941,0.127059
2,Residual,13.250000,81,NaN,NaN,NaN


In [450]:
ancova.to_csv('data/stats/ANCOVA_df_results.csv', index=None, encoding='utf-8')

In [445]:
df5

,Phases,Temperature,Respiration Rate,Vitals Heart Rate,Blood Pressure Systolic,Blood Pressure Diastolic,Mean Arterial Pressure,Step Count,Resting Heart Rate,Number of Floors,Heart Rate Count,Heart Rate Mean,Heart Rate STD,Heart Rate Minimum,Heart Rate 25%,Heart Rate Median,Heart Rate 75%,Heart Rate Maximum,Mood,Pain,Tired,Sick,Anxiousness,Depressed,Suicidal,Self-Harm,Thoughts of Death,Repetitive Thoughts,Thoughts of Worthlessness,Thoughts of Hopelessness,Brain Sludge,Sluggish,Restless,Existential Crisis,Loss of Interest,Self Isolation,Paranoia,Elevated,Irritable/Angry,Loneliness,Libido,Weight,Weird,sleeping_durations_seconds,sleeping_counts,sleeping_durations_seconds_7_day_rolling_mean,internet_durations_seconds,internet_counts,internet_durations_seconds_7_day_rolling_mean,working_durations_seconds,working_counts,working_durations_seconds_7_day_rolling_mean,games_durations_seconds,games_counts,games_durations_seconds_7_day_rolling_mean,family_time_durations_seconds,family_time_counts,family_time_durations_seconds_7_day_rolling_mean,eating_durations_seconds,eating_counts,eating_durations_seconds_7_day_rolling_mean,programming_durations_seconds,programming_counts,programming_durations_seconds_7_day_rolling_mean,grooming_durations_seconds,grooming_counts,grooming_durations_seconds_7_day_rolling_mean,reading_durations_seconds,reading_counts,reading_durations_seconds_7_day_rolling_mean,gym_durations_seconds,gym_counts,gym_durations_seconds_7_day_rolling_mean,toilet_durations_seconds,toilet_counts,toilet_durations_seconds_7_day_rolling_mean,friends_time_durations_seconds,friends_time_counts,friends_time_durations_seconds_7_day_rolling_mean,studying_durations_seconds,studying_counts,studying_durations_seconds_7_day_rolling_mean,movies_tv_durations_seconds,movies_tv_counts,movies_tv_durations_seconds_7_day_rolling_mean,driving_durations_seconds,driving_counts,driving_durations_seconds_7_day_rolling_mean,walking_durations_seconds,walking_counts,walking_durations_seconds_7_day_rolling_mean,writing_durations_seconds,writing_counts,writing_durations_seconds_7_day_rolling_mean,cooking_durations_seconds,cooking_counts,cooking_durations_seconds_7_day_rolling_mean,class_durations_seconds,class_counts,class_durations_seconds_7_day_rolling_mean,cleaning_durations_seconds,cleaning_counts,cleaning_durations_seconds_7_day_rolling_mean,dishes_durations_seconds,dishes_counts,dishes_durations_seconds_7_day_rolling_mean,personal_durations_seconds,personal_counts,personal_durations_seconds_7_day_rolling_mean,dating_durations_seconds,dating_counts,dating_durations_seconds_7_day_rolling_mean,shopping_durations_seconds,shopping_counts,shopping_durations_seconds_7_day_rolling_mean,mycology_durations_seconds,mycology_counts,mycology_durations_seconds_7_day_rolling_mean,errands_and_appointments_durations_seconds,errands_and_appointments_counts,errands_and_appointments_durations_seconds_7_day_rolling_mean,club_social_group_durations_seconds,club_social_group_counts,club_social_group_durations_seconds_7_day_rolling_mean,drinking_durations_seconds,drinking_counts,drinking_durations_seconds_7_day_rolling_mean,gamedev_durations_seconds,gamedev_counts,gamedev_durations_seconds_7_day_rolling_mean,swimming_tanning_durations_seconds,swimming_tanning_counts,swimming_tanning_durations_seconds_7_day_rolling_mean,phone_durations_seconds,phone_counts,phone_durations_seconds_7_day_rolling_mean,keyboard_durations_seconds,keyboard_counts,keyboard_durations_seconds_7_day_rolling_mean,journaling_durations_seconds,journaling_counts,journaling_durations_seconds_7_day_rolling_mean,drawing_durations_seconds,drawing_counts,drawing_durations_seconds_7_day_rolling_mean,traveling__durations_seconds,traveling__counts,traveling__durations_seconds_7_day_rolling_mean,drumming_durations_seconds,drumming_counts,drumming_durations_seconds_7_day_rolling_mean,data_analysis_durations_seconds,data_analysis_counts,data_analysis_durations_seconds_7_day_rolling_mean,concert_event_durations_seconds,

In [428]:
pingouin.ancova(data=df5, dv='Mood', covar=columns, between='Phases')

/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  warnings.warn('covariance of constraints does not have full '
/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/statsmodels/base/model.py:1900: RuntimeWarning: invalid value encountered in true_divide
  F /= J
/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  warnings.warn('covariance of constraints does not have full '
/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  warnings.warn('covariance of constraints does not have full '
/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/statsmodels

,Source,SS,DF,F,p-unc,np2
0,Phases,0.051758,1,0.360142,0.550365,0.005119
1,Pain,0.023500,1,0.163520,0.687169,0.002331
2,Tired,0.617034,1,4.293414,0.041946,0.057790
3,Sick,0.015254,1,0.106138,0.745556,0.001514
4,Anxiousness,1.377399,1,9.584143,0.002822,0.120428
5,Depressed,0.498843,1,3.471023,0.066648,0.047243
6,Suicidal,1.522384,1,10.592970,0.001750,0.131438
7,Self-Harm,1.560025,1,10.854883,0.001548,0.134251
8,Thoughts of Death,1.458221,1,10.146514,0.002160,0.126600
9,Repetitive Thoughts,0.121918,1,0.848322,0.360191,0.011974


In [438]:
pingouin.ancova(data=df5, dv='Mood', covar=vitals, between='Phases')

,Source,SS,DF,F,p-unc,np2
0,Phases,0.058248,1,0.426752,0.515697,0.005975
1,Step Count,0.496204,1,3.635431,0.060609,0.048709
2,Resting Heart Rate,0.540761,1,3.961879,0.050392,0.052852
3,Number of Floors,0.504449,1,3.695840,0.058561,0.049479
4,Heart Rate Count,0.951928,1,6.974292,0.010162,0.089443
5,Heart Rate Mean,0.005717,1,0.041886,0.838423,0.000590
6,Heart Rate STD,0.010126,1,0.074188,0.786124,0.001044
7,Heart Rate Minimum,0.161790,1,1.185352,0.279951,0.016421
8,Heart Rate 25%,0.006700,1,0.049088,0.825293,0.000691
9,Heart Rate Median,0.024136,1,0.176829,0.675383,0.002484


In [429]:
pingouin.ancova(data=df5, dv='Mood', covar=df_counts, between='Phases')

/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  warnings.warn('covariance of constraints does not have full '
/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/statsmodels/base/model.py:1900: RuntimeWarning: invalid value encountered in true_divide
  F /= J
/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  warnings.warn('covariance of constraints does not have full '
/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  warnings.warn('covariance of constraints does not have full '
/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/statsmodels

,Source,SS,DF,F,p-unc,np2
0,Phases,0.226468,1,1.882263,0.175967,0.034933
1,sleeping_counts,0.434648,1,3.612523,0.062896,0.064959
2,internet_counts,0.160875,1,1.337091,0.252832,0.025069
3,working_counts,0.000157,1,0.001301,0.971362,0.000025
4,games_counts,0.287083,1,2.386059,0.128487,0.043873
5,family_time_counts,0.007345,1,0.061049,0.805818,0.001173
6,eating_counts,0.009878,1,0.082099,0.775612,0.001576
7,programming_counts,0.078077,1,0.648928,0.424165,0.012326
8,grooming_counts,0.186309,1,1.548485,0.218939,0.028917
9,reading_counts,0.101108,1,0.840347,0.363530,0.015904


In [430]:
pingouin.ancova(data=df5, dv='Mood', covar=df_durations_seconds, between='Phases')

/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  warnings.warn('covariance of constraints does not have full '
/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/statsmodels/base/model.py:1900: RuntimeWarning: invalid value encountered in true_divide
  F /= J
/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  warnings.warn('covariance of constraints does not have full '
/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  warnings.warn('covariance of constraints does not have full '
/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/statsmodels

,Source,SS,DF,F,p-unc,np2
0,Phases,0.088423,1,0.651275,0.423333,1.236959e-02
1,sleeping_durations_seconds,0.031806,1,0.234266,0.630411,4.484902e-03
2,internet_durations_seconds,0.166998,1,1.230018,0.272507,2.310759e-02
3,working_durations_seconds,0.071511,1,0.526709,0.471247,1.002746e-02
4,games_durations_seconds,0.303995,1,2.239058,0.140608,4.128129e-02
5,family_time_durations_seconds,0.305828,1,2.252561,0.139442,4.151990e-02
6,eating_durations_seconds,0.249937,1,1.840898,0.180707,3.419145e-02
7,programming_durations_seconds,0.190312,1,1.401729,0.241818,2.624876e-02
8,grooming_durations_seconds,0.018853,1,0.138860,0.710932,2.663266e-03
9,reading_durations_seconds,0.041618,1,0.306532,0.582189,5.860299e-03


In [431]:
pingouin.ancova(data=df5, dv='Mood', covar=df_durations_seconds_7_day_rolling_mean, between='Phases')

/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  warnings.warn('covariance of constraints does not have full '
/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/statsmodels/base/model.py:1900: RuntimeWarning: invalid value encountered in true_divide
  F /= J
/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  warnings.warn('covariance of constraints does not have full '
/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  warnings.warn('covariance of constraints does not have full '
/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/statsmodels

,Source,SS,DF,F,p-unc,np2
0,Phases,0.002486,1,0.026286,0.871833,0.000505
1,sleeping_durations_seconds_7_day_rolling_mean,0.084083,1,0.889112,0.350078,0.016811
2,internet_durations_seconds_7_day_rolling_mean,0.066827,1,0.706645,0.404409,0.013407
3,working_durations_seconds_7_day_rolling_mean,0.065457,1,0.692159,0.409235,0.013136
4,games_durations_seconds_7_day_rolling_mean,0.080754,1,0.853911,0.359717,0.016156
5,family_time_durations_seconds_7_day_rolling_mean,0.116866,1,1.235759,0.271403,0.023213
6,eating_durations_seconds_7_day_rolling_mean,0.007696,1,0.081375,0.776576,0.001562
7,programming_durations_seconds_7_day_rolling_mean,0.033242,1,0.351508,0.555831,0.006714
8,grooming_durations_seconds_7_day_rolling_mean,0.000123,1,0.001304,0.971328,0.000025
9,reading_durations_seconds_7_day_rolling_mean,0.023365,1,0.247069,0.621242,0.004729


In [453]:
ancova2 = pingouin.ancova(data=df5, dv='Mood', covar=columns + vitals +  df_counts + df_durations_seconds, between='Phases')
ancova2

/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  warnings.warn('covariance of constraints does not have full '
/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/statsmodels/base/model.py:1900: RuntimeWarning: invalid value encountered in true_divide
  F /= J
/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  warnings.warn('covariance of constraints does not have full '
/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  warnings.warn('covariance of constraints does not have full '
/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/statsmodels

,Source,SS,DF,F,p-unc,np2
0,Phases,0.060550,1,1.947139,0.257248,0.393589
1,Pain,0.043624,1,1.402828,0.321538,0.318620
2,Tired,0.152073,1,4.890295,0.113953,0.619786
3,Sick,0.024242,1,0.779550,0.442298,0.206255
4,Anxiousness,0.299333,1,9.625837,0.053189,0.762392
...,...,...,...,...,...,...
112,data_analysis_durations_seconds,NaN,1,NaN,NaN,NaN
113,concert_event_durations_seconds,NaN,1,NaN,NaN,NaN
114,biking_durations_seconds,NaN,1,NaN,NaN,NaN
115,dancing_durations_seconds,NaN,1,NaN,NaN,NaN


In [454]:
ancova2.to_csv('data/stats/ANCOVA2_df_results.csv', index=None, encoding='utf-8')

In [452]:
with pd.option_context('display.max_rows', None,):
    print(pingouin.ancova(data=df5, dv='Mood', covar=columns + vitals +  df_counts + df_durations_seconds, between='Phases'))

                                         Source        SS  DF          F  \
0                                        Phases  0.060550   1   1.947139   
1                                          Pain  0.043624   1   1.402828   
2                                         Tired  0.152073   1   4.890295   
3                                          Sick  0.024242   1   0.779550   
4                                   Anxiousness  0.299333   1   9.625837   
5                                     Depressed  0.531807   1  17.101635   
6                                      Suicidal  0.227181   1   7.305592   
7                                     Self-Harm  0.508396   1  16.348797   
8                             Thoughts of Death  0.170907   1   5.495968   
9                           Repetitive Thoughts  0.280123   1   9.008099   
10                    Thoughts of Worthlessness  0.011092   1   0.356697   
11                     Thoughts of Hopelessness  0.481506   1  15.484099   
12          

/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  warnings.warn('covariance of constraints does not have full '
/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/statsmodels/base/model.py:1900: RuntimeWarning: invalid value encountered in true_divide
  F /= J
/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  warnings.warn('covariance of constraints does not have full '
/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  warnings.warn('covariance of constraints does not have full '
/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/statsmodels